In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tuannguyenvananh/pokemon-dataset-with-team-combat")

print("Path to dataset files:", path)

100%|██████████| 276k/276k [00:00<00:00, 29.9MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/tuannguyenvananh/pokemon-dataset-with-team-combat/versions/6


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pokemon = pd.read_csv(path + "/pokemon.csv", encoding='latin1')
combat = pd.read_csv(path + "/combats.csv", encoding='latin1')

#Normalize numeric values
names = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']
for col in names:
    pokemon[col] = (pokemon[col] - pokemon[col].min()) / (pokemon[col].max() - pokemon[col].min())

pokemon = pd.get_dummies(pokemon, columns=['Type 1', ], dtype=int)
pokemon = pd.get_dummies(pokemon, columns=['Type 2', ], dtype=int)

#Dropping Features and encoding True/False
pokemon = pokemon.drop('Name', axis=1)
# pokemon = pokemon.drop('Type 2', axis=1) #Dropping type2 to reduce dimensionality of the data
pokemon = pokemon.drop('Generation', axis=1)
pokemon['Legendary'] = pokemon['Legendary'].astype(int)
y = combat['Winner']
combat=combat.drop('Winner', axis=1)

In [6]:
# Encode y as true or false. True if the first pokemon wins and false if the second pokemon wins.
for i in range(len(combat)):
    if y[i] == combat['First_pokemon'][i]:
        y[i]=1
    else:
        y[i]=0

In [7]:
cols = pokemon.drop('#', axis=1).columns
merged1 = combat.merge(pokemon, left_on='First_pokemon', right_on='#', how='left')
merged1 = merged1.drop(columns=['#'])
merged1.columns = ['First_pokemon', 'Second_pokemon'] + [col + '1' for col in cols]

# Step 3: Merge with pokemon data for Second_pokemon
merged2 = merged1.merge(pokemon, left_on='Second_pokemon', right_on='#', how='left')
merged2 = merged2.drop(columns=['#', 'First_pokemon', 'Second_pokemon'])
merged2.columns = merged1.columns.tolist()[2:] + [col + '2' for col in cols]

# Final DataFrame
df = merged2
df.head()

,HP1,Attack1,Defense1,Sp. Atk1,Sp. Def1,Speed1,Legendary1,Type 1_Bug1,Type 1_Dark1,Type 1_Dragon1,...,Type 2_Ghost2,Type 2_Grass2,Type 2_Ground2,Type 2_Ice2,Type 2_Normal2,Type 2_Poison2,Type 2_Psychic2,Type 2_Rock2,Type 2_Steel2,Type 2_Water2
0,0.192913,0.318919,0.200000,0.190217,0.142857,0.205714,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.354331,0.459459,0.297778,0.434783,0.519048,0.588571,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.212598,0.189189,0.355556,0.380435,0.404762,0.200000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.153543,0.189189,0.155556,0.326087,0.095238,0.085714,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.271654,0.297297,0.533333,0.570652,0.238095,0.285714,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [11]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
model = MLPRegressor(random_state=42, max_iter=10000, solver='adam')
model.fit(X_train, y_train)

model.predict(X_test)
model.score(X_test, y_test)


0.8444461039292592

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
model = MLPRegressor(random_state=42, max_iter=10000, solver='lbfgs')
model.fit(X_train, y_train)

model.predict(X_test)
model.score(X_test, y_test)

In [ ]:
# do hyperparameter tuning
# sending the analysis in the Discord is okay
# "more is better than less" want to be able to cut stuff
# get accurary, precision, MLP-specific accuracy scores
# put them all into a bar chart

# have everything pushed to GitHub by Sunday
# have collective evaluations and graphs done by the end of Monday
# report will be finished by 31st
# meet-up sunday June 1 at 3?
# finish demo videos, website, and presentation by 4th
#